# UNSW DataThon 2018
Sean and lmcmz Submition

### Import all the package that we used in the DataThon

In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from time import time
import pandas as pd
import numpy as np
%matplotlib inline

## Pre-processing


### Change the format of DataFrame

In [2]:
# Loading the data
userDf = pd.read_csv("User Data.csv")
bookDf=pd.read_csv("Books Information.csv")

#rename for coding
userDf.columns=["ID","User_Diff","B_17","B_18","Rate_17"]
bookDf.columns=["ID","Genre","Barcode","Diff","N_W","P_17","P_18","Place","N_S"]

# Convert the book strings into a list 
def convert2list(x):
    bookList=x.replace(" ","").split(",")
    return [int(i) for i in bookList]

userDf["B_17"] = userDf["B_17"].apply(convert2list)
userDf["B_18"] = userDf["B_18"].apply(convert2list)

### Building the query dictionary

In [3]:
# Building a bookDiff dictionary for querying the book difficulty 
bookDiffDict={}
def addToDict(x):
    bookDiffDict[x["ID"]]=x["Diff"]
bookDf.apply(addToDict,axis=1)

# Building a diffRateDict default dictionary for querying the rate list
# corresponding to its user and book difficulty 
diffRateDict={i+1:defaultdict(list) for i in range(5)}
def addDiffRate(x):
    bookList=x["B_17"]
    for book in bookList:
        bookDiff=bookDiffDict[book]
        userDiif=x["User_Diff"]
        diffRateDict[bookDiff][userDiif].append(x["Rate_17"])
userDf.apply(addDiffRate,axis=1)

# Calculating Mean Rate for its corresponding user and book difficulty 
diffMeanRate={ }
for bookDiff,userDiifD in diffRateDict.items():
    tempRates=[]
    tempDict={}
    for userDiif,Rates in userDiifD.items():
        tempRates.extend(Rates)
        tempDict[userDiif]=np.mean(Rates)
    m=np.mean(tempRates)
    updateD={ k:i-m for k,i in tempDict.items()}
    updateD["m"]=m
    diffMeanRate[bookDiff]=updateD

## Processing

### Calculate the mean rate for each book

In [4]:
def dictAppend(x):
    for i in x["B_17"]:
        r=x["Rate_17"]
        bookDict[i].append(r)
bookDict=defaultdict(list)
userDf.apply(dictAppend, axis=1)
bookMean={k:np.mean(i) for k,i in bookDict.items()}

### Calculate the user bias for each user

In [5]:
def userBias(x):
    bookList = x["B_17"]
    score = x["Rate_17"]
    userDiif=x["User_Diff"]
    rateList=[]
    for book in bookList:
        boofDiff=bookDiffDict[book]
        if book in bookMean:
            predict=bookMean[book]
        else:
            predict=diffMeanRate[boofDiff][userDiif]
        rateList.append(predict)
    predict = np.mean(rateList)
    return score - predict
userDf["userBias"]=userDf.apply(userBias,axis=1)

### Prediction

In [6]:
def predict(x):
    bookList = x["B_18"]
    bookList.extend(x["B_17"])
    userDiif=x["User_Diff"]
    rateList=[    ]
    for book in bookList:
        boofDiff=bookDiffDict[book]
        if book in bookMean:
            predict=bookMean[book]+ x["userBias"]
        else:
            print(book)
            predict=diffMeanRate[boofDiff][userDiif]+ x["userBias"]
        rateList.append((predict if predict<10 else 10) if predict>0 else 0)
    avePred = np.mean(rateList)
    return avePred
userDf["R_18"]=userDf.apply(predict,axis=1)

### Change the output format 

In [7]:
outputDf=pd.concat([userDf["ID"],userDf["R_18"]],axis=1)
outputDf.columns=["User ID","Average Rating (2018)"]
outputDf.to_csv("output.csv",index=False)